# Atlas 14 HMS-Equivalent Hyetographs

## Purpose

This notebook demonstrates using **Atlas14Storm** from hms-commander to generate hyetographs that match HEC-HMS "Specified Pattern" at **10^-6 precision**.

### Background

ras-commander now supports TWO methods for Atlas 14 hyetograph generation:

1. **StormGenerator** (Alternating Block Method):
   - Flexible peak positioning (0-100%)
   - Works with any DDF data source
   - **NOT HMS-equivalent** (~1% depth conservation tolerance)

2. **Atlas14Storm** (Official NOAA Temporal Distributions) - **NEW**:
   - Matches HEC-HMS "Specified Pattern" exactly
   - Uses official NOAA Atlas 14 temporal distribution curves
   - **10^-6 precision depth conservation**
   - Supports all 5 quartiles (First, Second, Third, Fourth, All Cases)

### When to Use Which Method

| Use Case | Method |
|----------|--------|
| HMS-equivalent workflows | **Atlas14Storm** |
| Official Atlas 14 temporal patterns | **Atlas14Storm** |
| Flexible peak positioning (0-100%) | **StormGenerator** |
| Non-HMS workflows | **StormGenerator** |

### HMS Equivalence Proof

Atlas14Storm's HMS equivalence was validated in hms-commander:
- **6 validation proofs** documented in `hms-commander/examples/08_atlas14_hyetograph_generation.ipynb`
- **10^-6 precision** confirmed through ground truth comparison
- **Exact depth conservation**: `abs(total - expected) < 0.000001 inches`

### Reference

- **NOAA Atlas 14**: https://hdsc.nws.noaa.gov/pfds/
- **Temporal Distributions**: https://hdsc.nws.noaa.gov/pub/hdsc/data/
- **HMS-Commander**: Atlas14Storm implementation and validation

## Setup

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = True  # Set to True for local development, False for pip package

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

from ras_commander.precip import Atlas14Storm, ATLAS14_AVAILABLE
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

if not ATLAS14_AVAILABLE:
    print("\n⚠️ WARNING: hms-commander not installed")
    print("Install with: pip install hms-commander")
    raise ImportError("Atlas14Storm requires hms-commander>=0.1.0")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configure output
pd.set_option('display.max_rows', 30)
pd.set_option('display.float_format', '{:.6f}'.format)  # 6 decimals for 10^-6 precision

## Example 1: Basic HMS-Equivalent Hyetograph

Generate a 100-year, 24-hour storm for Houston, TX using official NOAA temporal distributions.

In [ ]:
# =============================================================================
# STORM PARAMETERS - Houston, TX example
# =============================================================================

# Location: Houston, TX (Atlas 14 Volume 11 - Texas, Region 3)
STATE = "tx"
REGION = 3

# Storm parameters
TOTAL_DEPTH_INCHES = 17.9  # From NOAA Atlas 14 for Houston 100-yr, 24-hr
AEP_PERCENT = 1.0          # 1% AEP = 100-year storm
QUARTILE = "All Cases"     # Use "All Cases" for typical design

print(f"Location: Texas Region {REGION} (Houston area)")
print(f"Total Depth: {TOTAL_DEPTH_INCHES} inches (from Atlas 14)")
print(f"AEP: {AEP_PERCENT}% ({1/AEP_PERCENT*100:.0f}-year storm)")
print(f"Quartile: {QUARTILE}")

In [ ]:
# Generate HMS-equivalent hyetograph
print("Generating HMS-equivalent hyetograph...")

hyeto = Atlas14Storm.generate_hyetograph(
    total_depth_inches=TOTAL_DEPTH_INCHES,
    state=STATE,
    region=REGION,
    aep_percent=AEP_PERCENT,
    quartile=QUARTILE
)

print(f"\n✓ Generated {len(hyeto)} time steps (30-minute intervals)")
print(f"✓ Total depth: {hyeto.sum():.6f} inches")
print(f"✓ Depth conservation error: {abs(hyeto.sum() - TOTAL_DEPTH_INCHES):.9f} inches")

# Verify HMS equivalence (10^-6 precision)
assert abs(hyeto.sum() - TOTAL_DEPTH_INCHES) < 1e-6, "Depth conservation failed!"
print("\n✓ HMS EQUIVALENCE VERIFIED: Depth conserved at 10^-6 precision")

In [ ]:
# Create time array for plotting
time_hours = np.arange(len(hyeto)) * 0.5  # 30-minute intervals

# Find peak
peak_idx = np.argmax(hyeto)
peak_time = time_hours[peak_idx]
peak_value = hyeto[peak_idx]

print(f"Peak intensity: {peak_value:.4f} inches at hour {peak_time:.1f}")

In [ ]:
# Visualize hyetograph
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot 1: Incremental depths (hyetograph)
ax1.bar(time_hours, hyeto, width=0.4, alpha=0.7, color='steelblue', edgecolor='black')
ax1.axvline(peak_time, color='red', linestyle='--', linewidth=1, label=f'Peak at {peak_time:.1f} hr')
ax1.set_xlabel('Time (hours)', fontsize=11)
ax1.set_ylabel('Incremental Depth (inches)', fontsize=11)
ax1.set_title(f'{1/AEP_PERCENT*100:.0f}-Year, 24-Hour Storm - Houston, TX\nAtlas 14 HMS-Equivalent (10^-6 precision)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Cumulative depth
cumulative = np.cumsum(hyeto)
ax2.plot(time_hours, cumulative, 'b-', linewidth=2)
ax2.axhline(TOTAL_DEPTH_INCHES, color='red', linestyle='--', linewidth=1, label=f'Atlas 14 Total: {TOTAL_DEPTH_INCHES} in')
ax2.set_xlabel('Time (hours)', fontsize=11)
ax2.set_ylabel('Cumulative Depth (inches)', fontsize=11)
ax2.set_title('Cumulative Precipitation', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final cumulative depth: {cumulative[-1]:.6f} inches")
print(f"Expected (Atlas 14): {TOTAL_DEPTH_INCHES:.6f} inches")
print(f"Error: {abs(cumulative[-1] - TOTAL_DEPTH_INCHES):.9f} inches (< 10^-6)")

## Example 2: Comparing All 5 Quartiles

Atlas 14 provides 5 quartile options representing different temporal distributions based on historical storm data.

In [ ]:
# Generate hyetographs for all 5 quartiles
quartiles = ["First Quartile", "Second Quartile", "Third Quartile", "Fourth Quartile", "All Cases"]
hyetos = {}

print("Generating hyetographs for all quartiles...\n")

for q in quartiles:
    hyeto_q = Atlas14Storm.generate_hyetograph(
        total_depth_inches=TOTAL_DEPTH_INCHES,
        state=STATE,
        region=REGION,
        aep_percent=AEP_PERCENT,
        quartile=q
    )
    hyetos[q] = hyeto_q
    
    # Verify depth conservation
    error = abs(hyeto_q.sum() - TOTAL_DEPTH_INCHES)
    peak_time_q = np.argmax(hyeto_q) * 0.5
    
    print(f"{q:20s}: Peak at {peak_time_q:5.1f} hr, Depth error: {error:.9f} in")
    assert error < 1e-6, f"Depth conservation failed for {q}!"

print("\n✓ All quartiles conserve depth at 10^-6 precision")

In [ ]:
# Plot all quartiles
fig, ax = plt.subplots(figsize=(14, 7))

colors = ['blue', 'green', 'orange', 'red', 'black']
for (q, hyeto_q), color in zip(hyetos.items(), colors):
    ax.plot(time_hours, hyeto_q, linewidth=2, label=q, color=color, alpha=0.7)

ax.set_xlabel('Time (hours)', fontsize=12)
ax.set_ylabel('Incremental Depth (inches)', fontsize=12)
ax.set_title(f'Atlas 14 Temporal Distributions - All 5 Quartiles\n{1/AEP_PERCENT*100:.0f}-Year, 24-Hour Storm - Houston, TX', 
             fontsize=13, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Different quartiles represent different temporal patterns from historical data.")
print("'All Cases' is recommended for typical design applications.")

## Example 3: Multi-AEP Suite

Generate design storms for multiple AEP values (10-yr, 25-yr, 50-yr, 100-yr, 500-yr).

In [ ]:
# Multi-AEP analysis
aep_suite = {
    '10-year': {'aep': 10.0, 'depth': 10.5},   # Example depths from Atlas 14
    '25-year': {'aep': 4.0, 'depth': 13.2},
    '50-year': {'aep': 2.0, 'depth': 15.3},
    '100-year': {'aep': 1.0, 'depth': 17.9},
    '500-year': {'aep': 0.2, 'depth': 24.1},
}

results = []

print("Generating multi-AEP suite...\n")

for storm_name, params in aep_suite.items():
    hyeto_aep = Atlas14Storm.generate_hyetograph(
        total_depth_inches=params['depth'],
        state=STATE,
        region=REGION,
        aep_percent=params['aep'],
        quartile="All Cases"
    )
    
    peak_idx = np.argmax(hyeto_aep)
    peak_value = hyeto_aep[peak_idx]
    error = abs(hyeto_aep.sum() - params['depth'])
    
    results.append({
        'Storm': storm_name,
        'AEP (%)': params['aep'],
        'Target Depth (in)': params['depth'],
        'Generated Depth (in)': hyeto_aep.sum(),
        'Error (in)': error,
        'Peak (in)': peak_value,
        'Peak Time (hr)': peak_idx * 0.5
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Verify all passed
max_error = results_df['Error (in)'].max()
print(f"\nMaximum depth conservation error: {max_error:.9f} inches")
print(f"✓ All storms meet 10^-6 precision requirement" if max_error < 1e-6 else "✗ FAILED")

## Example 4: Exporting for HEC-RAS

Convert to formats suitable for HEC-RAS input (CSV, DSS).

In [ ]:
# Create DataFrame for export
hyeto_export = pd.DataFrame({
    'Time (hours)': time_hours,
    'Incremental Depth (inches)': hyeto,
    'Cumulative Depth (inches)': np.cumsum(hyeto)
})

print("Hyetograph data ready for export:")
print(hyeto_export.head(10))

# Export to CSV (uncomment to save)
# hyeto_export.to_csv('atlas14_100yr_24hr_houston.csv', index=False)
# print("\n✓ Exported to CSV: atlas14_100yr_24hr_houston.csv")

print("\nNote: For DSS export, use pydsstools or HEC-DSSVue to import the CSV.")

## Comparison: Atlas14Storm vs StormGenerator

This section demonstrates the key differences between the two methods.

In [ ]:
from ras_commander.precip import StormGenerator

print("Comparing Atlas14Storm vs StormGenerator...\n")

# Method 1: Atlas14Storm (HMS-equivalent)
hyeto_hms = Atlas14Storm.generate_hyetograph(
    total_depth_inches=TOTAL_DEPTH_INCHES,
    state=STATE,
    region=REGION,
    aep_percent=AEP_PERCENT,
    quartile="All Cases"
)

# Method 2: StormGenerator (Alternating Block)
gen = StormGenerator.download_from_coordinates(29.76, -95.37)  # Houston
hyeto_ab = gen.generate_hyetograph(
    ari=100,  # 100-year
    duration_hours=24,
    position_percent=50  # Centered peak
)

# Compare
print("Method Comparison:")
print(f"\nAtlas14Storm (HMS-equivalent):")
print(f"  Total depth: {hyeto_hms.sum():.6f} inches")
print(f"  Depth error: {abs(hyeto_hms.sum() - TOTAL_DEPTH_INCHES):.9f} inches (< 10^-6)")
print(f"  Method: Official NOAA temporal distributions")
print(f"  HMS equivalent: YES (10^-6 precision)")

print(f"\nStormGenerator (Alternating Block):")
print(f"  Total depth: {hyeto_ab['cumulative_depth'].iloc[-1]:.6f} inches")
print(f"  Depth error: {abs(hyeto_ab['cumulative_depth'].iloc[-1] - TOTAL_DEPTH_INCHES):.6f} inches")
print(f"  Method: Alternating block with flexible peak positioning")
print(f"  HMS equivalent: NO (~1% tolerance)")

print(f"\n{'='*60}")
print("RECOMMENDATION:")
print("Use Atlas14Storm for HMS-equivalent workflows requiring 10^-6 precision.")
print("Use StormGenerator for flexible peak positioning (0-100%).")
print(f"{'='*60}")

## Conclusion

This notebook demonstrated **Atlas14Storm** - the HMS-equivalent hyetograph generation method now available in ras-commander.

### Key Takeaways

1. **10^-6 Precision**: Atlas14Storm conserves depth at `abs(total - expected) < 0.000001 inches`
2. **HMS Equivalent**: Matches HEC-HMS "Specified Pattern" exactly
3. **Official NOAA Temporal Distributions**: Uses validated curves from Atlas 14
4. **5 Quartile Options**: First, Second, Third, Fourth, All Cases
5. **Proven Validation**: 6-proof validation in hms-commander

### When to Use Which Method

| Requirement | Use This Method |
|-------------|----------------|
| HMS-equivalent hyetographs | **Atlas14Storm** |
| Official Atlas 14 temporal patterns | **Atlas14Storm** |
| 10^-6 depth conservation | **Atlas14Storm** |
| Flexible peak positioning (0-100%) | **StormGenerator** |
| Non-HMS workflows | **StormGenerator** |

### See Also

- **StormGenerator Examples**: `720_atlas14_aep_events.ipynb`, `722_atlas14_multi_project.ipynb`
- **Validation**: `hms-commander/examples/08_atlas14_hyetograph_generation.ipynb` (6 proofs)
- **Documentation**: `ras_commander/precip/CLAUDE.md`
- **Integration Tests**: `tests/test_atlas14_integration.py`